# \__init_subclass__()

重写\__init_subclass__方法， 先看下效果

In [1]:
class Hook:

    def __init_subclass__(cls, **kwargs):
        print("__init_subclass__", cls, kwargs)


class A(Hook, name="satori", age=16):
    pass

__init_subclass__ <class '__main__.A'> {'name': 'satori', 'age': 16}


首先我们定义了一个Hook类，然后让A这个类继承它。**发现我们还没有没有实例化，而是在创建类的时候就有输出结果了。**

对于一个类，如果这个类被作为父类继承，那么会触发其内部的\__init_subclass__方法，这里的Hook被A继承，那么Hook中的\__init_subclass__就会被触发。而且看到，里面的cls，就是我们的A，也就是继承它的类，**kwargs，就是我们额外传递的参数。

但是我们发现，第一个参数不是self，而是cls，而且这个cls还不是我们的Hook，而是继承它的类。其实这个方法是隐式的被classmethod装饰了


有时候我们想控制类的生成过程，怎么办呢？显然可以通过元类的的方式，但是如果场景比较简单，也没必要使用元类。直接使用\__init_subclass__即可

In [2]:
class Hook:

    def __init_subclass__(cls, **kwargs):
        for k, v in kwargs.items():
            type.__setattr__(cls, k, v)


class A(Hook, name="satori", age=16):
    pass


a = A()

a.name, a.age

('satori', 16)

可以看到，我们在不使用元类的情况下，通过__init_subclass__实现了类的自定义过程。当然这比较简单，也可以实现更复杂的逻辑，在某些场景下，可以替代元类。

---

# 获取所有子类 

新型类（即objectPython中默认的from的子类）具有一种\__subclasses__返回子类的方法：

In [18]:
class Foo(object):
    pass


class Bar(Foo):
    pass


class Baz(Foo):
    pass


class Bing(Bar):
    pass


class Cing(Bing):
    pass

In [19]:
# 这是子类的名称
print([cls.__name__ for cls in Foo.__subclasses__()])

['Bar', 'Baz']


In [20]:
# 这是子类本身
print(Foo.__subclasses__())

[<class '__main__.Bar'>, <class '__main__.Baz'>]


In [21]:
# 确认确实将子类Foo列为其基础
for cls in Foo.__subclasses__():
    print(cls.__base__)

<class '__main__.Foo'>
<class '__main__.Foo'>


In [22]:
# 请注意，如果需要子类，则必须递归
def all_subclasses(cls):
    return set(cls.__subclasses__()).union(
        [s for c in cls.__subclasses__() for s in all_subclasses(c)])

print(all_subclasses(Foo))

{<class '__main__.Cing'>, <class '__main__.Bing'>, <class '__main__.Bar'>, <class '__main__.Baz'>}


请注意，如果尚未执行子类的类定义（例如，如果尚未导入子类的模块），则该子类尚不存在，\__subclasses__也找不到它。

---

# 通过字符串查找类对象

如果确实有一个表示类名称的字符串，并且想要查找该类的子类，有两步：找到给定名称的类，然后使用\__subclasses__上述方法查找子类。

In [23]:
globals()['Foo']

__main__.Foo

In [24]:
locals()['Foo']

__main__.Foo

如果该类可以位于任何模块中，则你的名称字符串应包含完全限定的名称- ‘pkg.module.Foo’而不是just ‘Foo’。使用importlib加载类的模块，然后获取相应的属性

In [36]:
import importlib

name = 'lxml.etree.HTML'
modname, _, clsname = name.rpartition('.')
modname, clsname

('lxml.etree', 'HTML')

In [35]:
mod = importlib.import_module(modname)
cls = getattr(mod, clsname)
cls

<cyfunction HTML at 0x00000242F2E58778>